In [1]:
from itertools import combinations
import numpy as np

## Simple Test Problem

In [7]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('./lab2/test_problem.npy')

## Common tests

In [9]:
problem = np.load('lab2/problem_r2_100.npy')

In [10]:
# Negative values?
np.any(problem < 0)

np.True_

In [11]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [11]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [12]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False